# Bottleneck Floyd-Warshall TSP

In [1]:
%load_ext autoreload
%autoreload 2

## Get Spotify Tracks

In [2]:
import tekore as tk
from smoothify.features import construct_features
import numpy as np

### Obtaining an API Access Token

You'll need an access token to call the Spotify API.

Here's how to get a temporary token to play around with (expires in ~30 mins):

- Visit https://developer.spotify.com/console/get-playlist-tracks/
- Click 'Get Token'
- Select the following permissions:
    - `user-library-read`
    - `playlist-read-private`
    - `playlist-read-collaborative`
    - `playlist-modify-public`
    - `playlist-modify-private`
- Click 'Request Token'
- Copy the text in the 'OAuth Token' field

Paste the access token below

In [3]:
ACCESS_TOKEN = "BQA8KhAAWwCusuRxVGlixEilSgHfWATvhcFidd9S-MX_ZM0Mr5m2c0WTNeOP7Qzjppjytpq2neIbVXqAfqTTAe2T51EKozCHT3VthPlcYNI99vqFxbcX4xbHAgJdFScholbZtvdmhMIK4Zi2-6Aq2YsqPXj_McbiY8tHEwHCaipHtLTn7RCqlUeZI3c4Pga05EyQz8pLI0g8N1hMCPDMje0wppqUAF6NSiheJH_UXKUnQWUvoIE"
spotify = tk.Spotify(ACCESS_TOKEN, asynchronous=True, max_limits_on=True, chunked_on=True)
current_user = await spotify.current_user()

### Fetch tracks from current user's Liked Songs

In [4]:
saved_tracks_page = await spotify.saved_tracks()
track_list = [track.track async for track in spotify.all_items(saved_tracks_page)]
playlist_name = f"{current_user.display_name}'s Library"
is_playlist_public = False
is_playlist_collaborative = False
print(f"Fetched {len(track_list)} tracks")

Fetched 531 tracks


### Fetch tracks from a playlist

To read the tracks in a playlist, you'll need the playlist's ID.

You can find a playlist's ID in its URL, which looks like `https://open.spotify.com/playlist/{playlist_id}`

#### Getting a Playlist's URL

- Navigate to your playlist
- Under the playlist's title, click the triple-dot icon and select to `Share -> Copy link to playlist`


In [ ]:
PLAYLIST_ID = "78RmalWaNX12cJSmfVHlXL"
playlist = await spotify.playlist(PLAYLIST_ID)
track_list = [track.track async for track in spotify.all_items(playlist.tracks)]
playlist_name = playlist.name
is_playlist_public = playlist.public
is_playlist_collaborative = playlist.collaborative
print(f"Fetched {len(track_list)} tracks")

## Construct audio features

### Query Spotify API for audio features

In [5]:
# Extract track IDs so we can fetch additional info
track_id_list = [track.id for track in track_list]
# Get track audio features
track_features_list = await spotify.tracks_audio_features(track_id_list)

### Merge & normalize features

In [7]:
features_df = construct_features(audio_features_list=track_features_list)
features_df.head()

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,1.526728,-0.558664,-0.349908,0.130886,-0.678244,-0.738907,-0.702520,1.075576,-0.019850
1,-0.270316,1.331587,0.874591,0.340775,-0.564689,-0.784524,-0.357060,1.306981,0.268654
2,0.836608,0.242758,0.155296,-0.756161,-0.296462,-0.784524,1.909257,0.596806,0.184870
3,0.440302,1.187884,0.869561,1.564870,-0.644971,-0.784497,1.063982,0.405299,0.104714
4,0.993764,-0.354163,0.046521,-0.474267,-0.070465,-0.515523,-0.516315,-1.098835,-0.020427


In [8]:
points = np.array(features_df)

### Compute pairwise distances

In [107]:
import numpy as np
import scipy.spatial
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import floyd_warshall


class BottleneckFloydWarshallTSP:
    def __init__(self, *, points: np.ndarray, norm_power: float = 0.5):
        self.points = points
        self.norm_power = norm_power
        self.distance_matrix = self.pairwise_distances(self.points, norm_power=self.norm_power)

    def best_path(self):
        shortest_distances, predecessors = self.floyd_warshall(self.distance_matrix)
        shortest_distances[shortest_distances == 0] = np.inf
        best_src, best_dst = np.unravel_index(shortest_distances.argmin(), shortest_distances.shape)

        best_path = []
        idx = best_dst
        for i in range(len(self.points)):
            
        return shortest_distances

    @classmethod
    def pairwise_distances(cls, points: np.ndarray, norm_power: float = 0.5):
        """
        Given an array with shape (n_points, n_features), compute the pairwise distance matrix.
        """
        return scipy.spatial.distance.squareform(scipy.spatial.distance.pdist(points, "minkowski", p=norm_power))

    @classmethod
    def floyd_warshall(cls, distance_matrix: np.ndarray):
        shortest_distances, predecessors = floyd_warshall(distance_matrix, directed=False, return_predecessors=True)
        return shortest_distances, predecessors

In [108]:
optimizer = BottleneckFloydWarshallTSP(points=points)
best_path = optimizer.best_path()

[[-9999     0   191 ...     0     0     0]
 [    1 -9999     1 ...    69    70     1]
 [  191     2 -9999 ...   246   131   461]
 ...
 [  528    69   135 ... -9999   528   528]
 [  529    70   131 ...   529 -9999   529]
 [  530   530   461 ...   530   530 -9999]]


In [115]:
import seaborn as sns
sns.set_theme()
distance_matrix = BottleneckFloydWarshallTSP.pairwise_distances(points)
shortest, predecessors = BottleneckFloydWarshallTSP.floyd_warshall(distance_matrix)
# sns.displot(shortest.flatten())
shortest[shortest == 0] = np.inf
best_src, best_dst = np.unravel_index(shortest.argmin(), shortest.shape)
print(best_src, best_dst)
predecessors[best_dst, best_src]

106 173


173